In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [78]:
df = pd.read_csv('box_data.csv')
df.head()

,production_code,voice,chalkboard_gag,couch_gag,show_runner,written_by,directed_by
0,7G08,NaN,NaN,NaN,"James L. Brooks, Matt Groening, Sam Simon",Mimi Pond,David Silverman
1,7G02,"Marcia Wallace, Ms. Mellon","""I will not waste chalk""",Squashed Bart couch gag,"James L. Brooks, Matt Groening, Sam Simon",Jon Vitti,David Silverman
2,7G03,Marcia Wallace,"""I will not skateboard in the halls""",llapsed Couch couch gag,"James L. Brooks, Matt Groening, Sam Simon","Jay Kogen, Wallace Wolodarsky",Wes Archer
3,7G04,NaN,"""I will not burp in class""",Squashed Homer couch gag,"James L. Brooks, Matt Groening, Sam Simon","Al Jean, Mike Reiss","Gregg Vanzo, Kent Butterworth"
4,7G05,NaN,This episode did not feature a chalkboard gag,NaN,"James L. Brooks, Matt Groening, Sam Simon",John Swartzwelder,David Silverman


In [79]:
# count occurences of each value in a column
show_runner_list = df['show_runner'].str.split(',').explode('show_runner').value_counts()
couch_gag_list = df['couch_gag'].str.lower().value_counts()
guest_speaker_list =df['voice'].str.split(',').explode('voice').str.strip().value_counts()
chalkboard_gag_list = df['chalkboard_gag'].str.lower().value_counts()
written_by_list = df['written_by'].str.split(',').explode('written_by').str.strip().value_counts()
directed_by_list = df['directed_by'].str.split(',').explode('directed_by').str.strip().value_counts()

In [80]:
df_full = df.copy()
# delete rows with nan values in the column 'voice'
df = df.dropna(subset=['voice'])
df['voice'] = df['voice'].str.split(',')
frame = df[['production_code', 'voice']].explode('voice')

In [81]:
# create a list for each guest speaker that stores the production code of each episdoe he/she was in
speaker_production_code = []
for guest in frame['voice'].unique():
    speaker_production_code.append(frame[frame['voice'] == guest]['production_code'].tolist())

# create a dictionary that stores the guest speaker as the key and the list of production codes as the value
guest_speaker_dict = {}
for i in range(len(speaker_production_code)):
    guest_speaker_dict[frame['voice'].unique()[i]] = speaker_production_code[i]

In [91]:
df_episodes = pd.read_csv('Datasets/data-society-the-simpsons-by-the-data/simpsons_episodes.csv')
df_episodes[['imdb_rating','imdb_votes']] = df_episodes[['imdb_rating','imdb_votes']].fillna(0)
df_episodes[['imdb_rating', 'imdb_votes']] = df_episodes[['imdb_rating', 'imdb_votes']].replace('', 0)


df_guest_speaker_rating = pd.DataFrame(columns=['guest_speaker', 'imdb_rating', 'imd_votes', 'number_of_episodes'])
for i in range(len(speaker_production_code)):
    
    episodes_speaker = guest_speaker_dict[frame['voice'].unique()[i]]
    guest_speaker = frame['voice'].unique()[i]
    num_episodes = len(episodes_speaker)
    episodes_rating = []
    episodes_votes = []

    for episode in episodes_speaker:
        #only take the episodes that we have the data from in the kaggle csv sheet
        if episode not in df_episodes['production_code'].unique():
            continue

        rating = df_episodes.loc[df_episodes['production_code'] == episode, 'imdb_rating'].values[0]
        votes = df_episodes.loc[df_episodes['production_code'] == episode, 'imdb_votes'].values[0]
        episodes_rating.append(rating)
        episodes_votes.append(votes)

    df_guest_speaker_rating.loc[i] = [guest_speaker, episodes_rating, episodes_votes, num_episodes]

In [93]:
df_guest_speaker_rating.to_csv('guest_speaker_rating.csv')
df_guest_speaker_rating.head(20)

,guest_speaker,imdb_rating,imd_votes,number_of_episodes
0,Marcia Wallace,"[7.8, 7.5, 8.2, 8.3, 8.4, 8.4, 8.2, 8.4, 8.2, ...","[1973.0, 1709.0, 1369.0, 1272.0, 1414.0, 1394....",25
1,Ms. Mellon,[7.8],[1973.0],1
2,Ron Taylor,"[7.6, 8.3]","[1674.0, 1084.0]",2
3,Albert Brooks,"[7.9, 7.5, 6.0, 9.2, 7.2, 7.3, 6.8]","[1638.0, 1578.0, 1051.0, 2439.0, 562.0, 505.0,...",7
4,Sam McMurray,[7.4],[1511.0],1
5,Kelsey Grammer,"[8.3, 8.2, 9.0, 8.3, 8.2, 7.2, 7.3, 7.1, 6.8, ...","[1716.0, 1233.0, 2010.0, 1142.0, 1100.0, 668.0...",13
6,Penny Marshall,[7.9],[1567.0],1
7,Christopher Collins,[7.9],[1567.0],1
8,June Foray,[7.9],[1567.0],1
9,Paul Willson,[7.9],[1567.0],1


In [137]:
df_scripts = pd.read_csv('Datasets/data-society-the-simpsons-by-the-data/simpsons_script_lines.csv')
df_scripts.head()

C:\Users\Jan\AppData\Local\Temp/ipykernel_8616/3301678477.py:1: DtypeWarning: Columns (4,5,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_scripts = pd.read_csv('Datasets/data-society-the-simpsons-by-the-data/simpsons_script_lines.csv')


,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,9549,32,209,"Miss Hoover: No, actually, it was a little of ...",848000,True,464.0,3.0,Miss Hoover,Springfield Elementary School,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9550,32,210,Lisa Simpson: (NEAR TEARS) Where's Mr. Bergstrom?,856000,True,9.0,3.0,Lisa Simpson,Springfield Elementary School,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9551,32,211,Miss Hoover: I don't know. Although I'd sure l...,856000,True,464.0,3.0,Miss Hoover,Springfield Elementary School,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9552,32,212,Lisa Simpson: That life is worth living.,864000,True,9.0,3.0,Lisa Simpson,Springfield Elementary School,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9553,32,213,Edna Krabappel-Flanders: The polls will be ope...,864000,True,40.0,3.0,Edna Krabappel-Flanders,Springfield Elementary School,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
df_scripts = df_scripts.dropna(subset=['character_id','word_count'])
df_scripts['character_id'] = df_scripts['character_id'].astype(int)
df_scripts['word_count'] = df_scripts['word_count'].astype(int)
df_scripts = df_scripts[['character_id', 'word_count','episode_id']]	
len(df_scripts.index)

id                      int64
episode_id              int64
number                  int64
raw_text               object
timestamp_in_ms        object
speaking_line          object
character_id          float64
location_id           float64
raw_character_text     object
raw_location_text      object
spoken_words           object
normalized_text        object
word_count            float64
Unnamed: 13           float64
Unnamed: 14           float64
Unnamed: 15           float64
Unnamed: 16           float64
Unnamed: 17           float64
Unnamed: 18           float64
Unnamed: 19           float64
Unnamed: 20           float64
Unnamed: 21           float64
Unnamed: 22           float64
Unnamed: 23           float64
Unnamed: 24           float64
Unnamed: 25           float64
Unnamed: 26            object
dtype: object


C:\Users\Jan\AppData\Local\Temp/ipykernel_8616/22943922.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scripts['character_id'] = df_scripts['character_id'].astype(int)
C:\Users\Jan\AppData\Local\Temp/ipykernel_8616/22943922.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scripts['word_count'] = df_scripts['word_count'].astype(int)


132095

In [154]:
# create a list for each episode that stores the character id and the word count of each character in the episode
episode_id = []
for episode in df_scripts['episode_id'].unique():
    episode_id.append(df_scripts[df_scripts['episode_id'] == episode][['character_id','word_count']].values)

# create a dictionary that stores the episode id as the key and a list of tuples that represent the character id and the word count as the value
episode_dict = {}
for i in range(len(episode_id)):
    episode_dict[df_scripts['episode_id'].unique()[i]] = episode_id[i]

In [155]:
episode_dict[81]

array([[  33,   11],
       [  33,    4],
       [ 211,    7],
       [1113,    7],
       [ 211,   16],
       [1114,   17],
       [ 211,    3],
       [1114,    4],
       [ 211,    1],
       [1115,   21],
       [1093,   13],
       [1114,    7],
       [1116,   18],
       [   2,    3],
       [  33,    3],
       [   8,    4],
       [   2,    1],
       [   8,    2],
       [   2,   10],
       [   9,    9],
       [   2,   13],
       [  15,    5],
       [  14,    3],
       [ 140,   18],
       [ 273,    4],
       [   9,    9],
       [   2,    6],
       [1117,    3],
       [1118,    4],
       [1117,   14],
       [1118,   14],
       [   9,    8],
       [   8,   16],
       [1117,    5],
       [   8,    2],
       [1117,   14],
       [1117,   11],
       [ 620,   17],
       [1117,    5],
       [1117,    5],
       [1119,    4],
       [1120,    4],
       [ 658,    7],
       [ 139,   24],
       [ 595,    5],
       [ 139,    7],
       [ 139,    7],
       [ 139,